In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
import tensorflow as Graph
Graph.device("/device:CPU:0")
import numpy as np

import h5py
import time
import cv2

import matplotlib.pylab as plt

import keras.backend as K
from keras.utils import generic_utils
from keras.optimizers import Adam, SGD

from keras.models import Model
from keras.layers.core import Flatten, Dense, Dropout, Activation, Lambda, Reshape
from keras.layers.convolutional import Conv2D, Deconv2D, ZeroPadding2D, UpSampling2D
from keras.layers import Input, Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
import keras.backend as K
from keras.models import load_model



%matplotlib inline

Using TensorFlow backend.


In [2]:
def normalization(X):
    return X / 127.5 - 1

def load_data(datasetpath):
    with h5py.File(datasetpath, "r") as hf:
        X_full_train = hf["train_data_raw"][:].astype(np.float32)
        X_full_train = normalization(X_full_train)
        X_sketch_train = hf["train_data_gen"][:].astype(np.float32)
        X_sketch_train = normalization(X_sketch_train)
        X_full_val = hf["val_data_raw"][:].astype(np.float32)
        X_full_val = normalization(X_full_val)
        X_sketch_val = hf["val_data_gen"][:].astype(np.float32)
        X_sketch_val = normalization(X_sketch_val)
        return X_full_train, X_sketch_train, X_full_val, X_sketch_val

In [3]:
def train():
    print("load data")
    rawImage, procImage, rawImage_val, procImage_val = load_data(datasetpath)

    img_shape = rawImage.shape[-3:]
    patch_num = (img_shape[0] // patch_size) * (img_shape[1] // patch_size)
    disc_img_shape = (patch_size, patch_size, procImage.shape[-1])


    print("load generator model")
    generator_model = load_model(generator_path)

    use = rawImage_val
    use2 = rawImage
 
    imgs = generator_model.predict(use)
    for i,img in enumerate(imgs):
        img = inverse_normalization(img)
        plt.imshow(img)
        plt.axis('off')
        plt.savefig("output/testData/"+str(i)+".jpg")
        plt.clf()
        plt.close()
        
    imgs = generator_model.predict(use2)
    for i,img in enumerate(imgs):
        img = inverse_normalization(img)
        plt.imshow(img)
        plt.axis('off')
        plt.savefig("output/T_Data/"+str(i)+".jpg")
        plt.clf()
        plt.close()
        
    plt.imshow(use[0])
    plt.axis('off')
    plt.savefig("output/T_Data/exem0.jpg")
    plt.clf()
    plt.close()

In [4]:
def l1_loss(y_true, y_pred):
    return K.sum(K.abs(y_pred - y_true), axis=-1)

def inverse_normalization(X):
    return (X + 1.) / 2.

def to3d(X):
    if X.shape[-1]==3: return X
    b = X.transpose(3,1,2,0)
    c = np.array([b[0],b[0],b[0]])
    return c.transpose(3,1,2,0)

In [5]:
datasetpath = './output/datasetimages.hdf5'
generator_path = "backup/e_1000,64x64/output/generator_data/generator_model.h5"
patch_size = 32
batch_size = 16
epoch = 1000
train()
print("END")

load data
load generator model
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


END
